In [1]:
# !pip install flash_attn==2.5.8
!pip install torch accelerate transformers



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install tiktoken==0.8.0 chromadb==0.6.2 FlagEmbedding==1.3.3 pymupdf4llm==0.0.17 tenacity==9.0.0 python-dotenv==1.0.1

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
checkpoint = "meta-llama/Llama-3.2-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from chromadb.utils import embedding_functions
from FlagEmbedding import FlagReranker
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from FlagEmbedding import BGEM3FlagModel
import pymupdf4llm
import logging
from tqdm import tqdm
import numpy as np

logger = logging.getLogger(__name__)

In [ ]:
def embedding_function_bge(text_list):
    return emb_model.encode(text_list, return_dense=True)['dense_vecs']



class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = embedding_function_bge(input)
        return embeddings

emb_model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
default_ef = MyEmbeddingFunction()
client = chromadb.PersistentClient(path="chromadb_folder")

In [167]:
def process_texts(texts, chunk_size=100, overlap=30):
    """Process a list of texts, splitting them into chunks of specified size with overlap,
    and accumulating shorter texts."""
    accumulated_words = []  # Accumulate words from texts shorter than chunk_size
    final_chunks = []  # Store the final chunks of text

    for text in texts.split():
        accumulated_words.append(text)

        while len(accumulated_words) >= chunk_size:
            # Take the first chunk_size words for the current chunk
            chunk = " ".join(accumulated_words[:chunk_size])
            final_chunks.append(chunk)
            # Remove words from the start of the accumulated_words, considering overlap
            accumulated_words = accumulated_words[chunk_size - overlap:]

    # If there are any remaining words, form the last chunk
    if accumulated_words:
        final_chunks.append(" ".join(accumulated_words))

    return final_chunks

def get_unique_text_indices(text_list):
    unique_texts = {}
    unique_indices = []

    for i, text in enumerate(text_list):
        if text not in unique_texts:
            unique_texts[text] = i
            unique_indices.append(i)

    return unique_indices

def create_pdf_collection(pdf_path):
    """
    Process a PDF file and add its chunks to a collection.

    Args:
        pdf_path: Path to the PDF file
        collection: The collection object to add documents to
    """
    try:
        md_text = pymupdf4llm.to_markdown(pdf_path,show_progress=True)
        all_chunks = process_texts(md_text, chunk_size=500, overlap=50)
        collection = client.get_or_create_collection(name='database',embedding_function=default_ef)
        logger.info(collection)

        for idx, chunk in tqdm(enumerate(all_chunks)):
            id_ = str(idx)
            collection.add(
                documents=[chunk],
                ids=[id_]
            )
        status = 'success'
        return status,collection_name

    except Exception as e:
        logger.error(f"Error creating pdf collection: {str(e)}", exc_info=True)
        return f'Sorry for inconvenience. Error creating pdf collection. Please contact support.'


def get_full_context(query, collection, n_results=5, top=2):

    ''''
    Get the context from database for given query.

    Args:
        query: Query string
        collection_name: Database collection object
        n_results: Number of results to retrieve
        top: Number of top results to return
    Returns:
          str: context based on query.
    '''

    logger.info(f'quering collection---> {collection}')

    result = collection.query(query_texts = query,n_results=n_results)
    texts = result['documents'][0]
    ids = result['ids'][0]
    unique_indices = get_unique_text_indices(texts)
    unique_docs = [texts[x] for x in unique_indices]
    unique_ids = [ids[x] for x in unique_indices]
    ## colbert
    query_col = emb_model.encode([query],return_colbert_vecs=True)
    docs_col = emb_model.encode(unique_docs,return_colbert_vecs=True)
    colber_scores = []
    for vectors in docs_col['colbert_vecs']:
        colber_scores.append(emb_model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())

    ## full_context_colbert
    full_context_scores = []
    full_context_ids = []
    for id in unique_ids:
        pre_id,post_id = str(int(id)-1), str(int(id)+1)
        # print(pre_id,id,post_id)
        full_context_ids.append([pre_id,id,post_id])
        full_context=collection.get(ids=[f'{pre_id}',f'{id}',f'{post_id}'])['documents']
        full_context = ''.join(full_context)
        full_context_colber_vec = emb_model.encode([full_context],return_colbert_vecs=True)
        full_context_colber_score = emb_model.colbert_score(query_col['colbert_vecs'][0],full_context_colber_vec['colbert_vecs'][0]).numpy()

        full_context_scores.append(full_context_colber_score)

    all_scores = [2*full_context_scores[i]+0.9*colber_scores[i] for i in range(len(colber_scores))]
    sorted_indices = [index for index, _ in sorted(enumerate(all_scores), key=lambda x: x[1], reverse=True)]
    top_context_ids_list = [full_context_ids[index] for index in sorted_indices][:top]
    flattened_list = np.array(top_context_ids_list).flatten().tolist()
    top_ids = list(set(flattened_list))
    top_ids.sort()
    top_context = collection.get(ids=top_ids)['documents']
    # top_context = ''.join(top_context)

    logger.info(f'context retrieved from collection---> {collection}')
    return top_context,top_ids


In [168]:
ll = [3,6,2,8]
ll.sort()
ll

[2, 3, 6, 8]

In [169]:
create_pdf_collection('/workspace/llama3-report.pdf')

Processing /workspace/llama3-report.pdf...
[                                        ] (0/92[                                        ] ( 1/9[                                        ] ( 2/92[=                                       ] ( 3/[=                                       ] ( 4/9[==                                      ] ( 5/9[==                                      ] ( 6/92[===                                     ] ( 7/[===                                     ] ( 8/[===                                     ] ( 9/9[====                                    ] (10/9[====                                    ] (11/92[=====                                   ] (12/[=====                                   ] (13/9[======                                  ] (14/9[======                                  ] (15/9[======                                  ] (16/92[=======                                 ] (17/[=======                                 ] (18/9[========                                ] (19

119it [00:13,  8.58it/s]


('success', 'llama3-report.pdf')

In [10]:
# collection_name = 'llama3-report.pdf'
# collection = client.get_or_create_collection(name=collection_name,embedding_function=default_ef)


In [ ]:
# query = 'what is training algorithms used here'
# get_full_context(query, collection, n_results=5, top=2)

In [170]:
def generate(prompt, model, tokenizer, add_special_tokens=True):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=add_special_tokens,
    ).to(model.device)

    with torch.no_grad():
      outputs = model.generate(**inputs, max_new_tokens=150, do_sample=True)

    result = outputs[0][inputs["input_ids"].shape[-1]:-1]
    return tokenizer.decode(result, skip_special_tokens=False)


In [171]:
inputs = tokenizer(
        'hi there',
        return_tensors="pt",
        add_special_tokens=True,
    ).to(model.device)
inputs["input_ids"].shape[-1]

3

In [276]:
import inspect

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "dict",
                "properties": parameters,
                "required": required,
            },
        }
    

In [ ]:
function_definitions = """[
    {
        "name": "get_user_info",
        "description": "Retrieve details for a specific user by their unique identifier. Note that the provided function is in Python 3 syntax.",
        "parameters": {
            "type": "dict",
            "required": [
                "user_id"
            ],
            "properties": {
                "user_id": {
                "type": "integer",
                "description": "The unique identifier of the user. It is used to fetch the specific user details from the database."
            },
            "special": {
                "type": "string",
                "description": "Any special information or parameters that need to be considered while fetching user details.",
                "default": "none"
                }
            }
        }
    }
]
"""

In [275]:
function_to_schema(get_full_context)

query
collection
n_results=5
top=2


{'name': 'get_full_context',
 'description': "'\n    Get the context from database for given query.\n\n    Args:\n        query: Query string\n        collection_name: Database collection object\n        n_results: Number of results to retrieve\n        top: Number of top results to return\n    Returns:\n          str: context based on query.",
 'parameters': {'type': 'dict',
  'properties': {'query': {'type': 'string'},
   'collection': {'type': 'string'},
   'n_results': {'type': 'string'},
   'top': {'type': 'string'}},
  'required': ['query', 'collection']}}

In [174]:
import json
# def execute_tool_call(tool_call, tools, agent_name):
#     name = tool_call.function.name
#     args = json.loads(tool_call.function.arguments)

#     print(f"{agent_name}:", f"{name}({args})")

#     return tools[name](**args)

In [182]:
def get_answer(query,context):

  '''
  Input: query, context
  Return the answer based on context for given query.
  '''
  prompt = f'''<|start_header_id|>system<|end_header_id|>\
  Analyze the given context {context} and answer this {query}. Write you answer in concise words if context is enough,otherwise say user you don't have enough context to answer<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'''
  response = generate(prompt, model, tokenizer, add_special_tokens=True)

  return response


def get_context_based_on_collection(query):
  '''
  Input: query, collection name
  Return the context based on collection of pdf file for given query.
  '''
  collection = client.get_or_create_collection(name='database',embedding_function=default_ef)

  context, context_idx = get_full_context(query, collection, n_results=5, top=2)

  if context:
    print('context found')
    return ' '.join(context)
  else:
    return f'No context found for given {query} through collection {collection_name}'

def get_answer_based_on_collection(query):
  '''
  Input: query, context based on index of file
  Return the answer based on context for given query.
  '''

  context = get_context_based_on_collection(query)
  answer = get_answer(query,context)
  return answer



In [183]:
function_to_schema(get_answer_based_on_collection)

{'type': 'function',
 'function': {'name': 'get_answer_based_on_collection',
  'description': 'Input: query, context based on index of file\n  Return the answer based on context for given query.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string'}},
   'required': ['query']}}}

In [176]:
get_answer('what is my name',' My name is rohit and i an engineer')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Your name is Rohit.'

In [177]:
from pydantic import BaseModel
from typing import Optional

class Agent(BaseModel):
    name: str = "Agent"
    model: object = None
    tokenizer: object = None
    instructions: str = "You are a helpful Agent"
    tools: list = []

class Response(BaseModel):
    agent: Optional[Agent]
    messages: str



In [324]:
def transfer_to_answer_agent():
  '''
  Return the answering agent.

  '''
  return answer_agent


def transfer_to_main_agent():
  '''
  Return the main agent.

  '''
  return main_agent

main_agent = Agent(
    name="Main Agent",
    instructions = '''
You are a helpful Agent. Your primary task is to provide answers to user queries based on database collections.
- Always greet the user during their first interaction and clearly state your purpose.
- Do not assume any role other than that of a helpful Agent.

### Workflow:
1. **Determine Query Type:** 
   - If the user’s query requires a collection-based answer, always delegate to the answer agent.
   - If the query lacks sufficient parameters for any available functions, point this out explicitly.
2. **Function Calls:** 
   - To achieve your goal, use one or more function/tool calls.
   - If none of the available functions can be used, explicitly state that no suitable function exists for the query.
   - Always ensure to return the function calls in the tools call section only.

### Function Call Format:
- When invoking functions, follow this exact format:
<python_tag=function_name(param_name1=param_value1, param_name2=param_value2...)/python_tag>
- Do not include any additional text outside of the `<python_tag>` structure in the tools call section.

### Example Usage:
- A valid function call:
<python_tag=get_data(query="user_query", limit=10)/python_tag>

### Additional Notes:
- If no function or tool can fulfill the query, clearly state this without including unnecessary text.
- Ensure responses strictly follow the format mentioned above.

Here is a list of functions in JSON format that you can invoke:
''',
    tools = [transfer_to_answer_agent]
)

answer_agent = Agent(
    name="Answering Agent",
    instructions='''
You are a helpful Agent. Your task is to answer user queries based on the context provided from the database collection. 
- If the collection-based answer is insufficient, revert to the main agent function.

### Workflow:
1. **Determine Query Type:** 
   - Analyze the user query to determine if it requires a collection-based answer.
   - If no suitable collection-based answer is found, point it out and return to the main agent.
2. **Function/Tool Calls:** 
   - To achieve the purpose of answering the query, you may invoke one or more functions/tools.
   - If none of the functions can be used, explicitly state this.
   - If the query lacks required parameters for a function, also point this out.

### Function Call Format:
- When invoking functions, use this exact format:
<python_tag=function_name(param_name1=param_value1, param_name2=param_value2...)/python_tag>
- Do not include any additional text outside of the `<python_tag>` structure in the tools call section.

### Example Usage:
- A valid function call:
<python_tag=get_data(query="user_query", limit=10)/python_tag>

### Additional Notes:
- If no function or tool can fulfill the query, clearly state this without including unnecessary text.
- Ensure responses strictly follow the format mentioned above.

Here is a list of functions in JSON format that you can invoke:
''',
    tools = [get_answer_based_on_collection, transfer_to_main_agent]

)

In [278]:
[function_to_schema(tool) for tool in main_agent.tools]

[{'name': 'transfer_to_answer_agent',
  'description': 'Return the answering agent.',
  'parameters': {'type': 'dict', 'properties': {}, 'required': []}}]

In [271]:
def run_full_turn(agent, user_msg, full_context):

    current_agent = agent
    
    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool)['function'] for tool in current_agent.tools]
        tools = {tool.__name__: tool for tool in current_agent.tools}

        # === 1. get llm reponse ===
        prior_system_instruction = current_agent.instructions
        custom_tool = tool_schemas
        system_prompt = f"<|start_header_id|>system<|end_header_id|>\n\n{prior_system_instruction}{custom_tool}<|eot_id|>"
        
        user_prompt = f"\n\n<|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n\n"
        
        prompt = f"{system_prompt}{full_context}{user_prompt}"
        
        # print('Full prompt:\n', prompt)
        
        response = generate(prompt, model, tokenizer, add_special_tokens=True)
        
        full_context = full_context+user_prompt+response+'<|eot_id|>'
        if response:
            print(f"{current_agent.name}:", response)
            print(f"tools avalable: {tools}")

        if 'function' not in response:
            break
        
        
        
        if 'function' in response:
            print('tool call detected')
            for res in [response]:
                
                result = execute_tool_call(res, tools)
                print('Tool call response: ', result)
        
                if type(result) is Agent:  # if agent transfer, update current agent
                    current_agent = result
                    result = (
                        f"Transfered to {current_agent.name}. Adopt persona immediately."
                    )
                
                full_context = full_context+f'<|eom_id|><|start_header_id|>ipython<|end_header_id|>{result}<|eot_id|>'


    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=full_context)


def execute_tool_call(res, tools):
    name, params, err_msg = parse_function_tag(res)
    print(f"executing tool:", f"{name}", ' parameters are: ',f'{params}')
    if err_msg=='success':
        try:
           return tools[name](**params)

        except Exception as e:
            return f"Error : {str(e)}"
    else:
        return err_msg


agent = main_agent
full_context = ''

while True:
    user_msg = input("User: ").strip()
    if user_msg.lower()=='q':
      break
    
    response = run_full_turn(agent, user_msg, full_context)
    agent = response.agent
    full_context = response.messages
    

User:  hi


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Main Agent: Hello! I'm an Agent, and my purpose is to provide information and answer questions based on the database collections I have access to. How can I assist you today?
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}


User:  tell me about training methods


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Main Agent: <function=answer_training_methods>{ "method1": "Positive Reinforcement", "method2": "Negative Reinforcement", "method3": "Operant Conditioning", "method4": "Imprinting", "method5": "Classical Conditioning", "method6": "Social Learning", "method7": "Cognitive Training", "method8": "Exposure Therapy" }</function>
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}
tool call detected
executing tool: answer_training_methods  parameters are:  {'method1': 'Positive Reinforcement', 'method2': 'Negative Reinforcement', 'method3': 'Operant Conditioning', 'method4': 'Imprinting', 'method5': 'Classical Conditioning', 'method6': 'Social Learning', 'method7': 'Cognitive Training', 'method8': 'Exposure Therapy'}
Tool call response:  Error : 'answer_training_methods'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Main Agent: <function=answer_training_methods>{ "method1": "Positive Reinforcement", "method2": "Negative Reinforcement", "method3": "Operant Conditioning", "method4": "Imprinting", "method5": "Classical Conditioning", "method6": "Social Learning", "method7": "Cognitive Training", "method8": "Exposure Therapy", "method9": "Desensitization", "method10": "Shaping" }</function>
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}
tool call detected
executing tool: answer_training_methods  parameters are:  {'method1': 'Positive Reinforcement', 'method2': 'Negative Reinforcement', 'method3': 'Operant Conditioning', 'method4': 'Imprinting', 'method5': 'Classical Conditioning', 'method6': 'Social Learning', 'method7': 'Cognitive Training', 'method8': 'Exposure Therapy', 'method9': 'Desensitization', 'method10': 'Shaping'}
Tool call response:  Error : 'answer_training_methods'


KeyboardInterrupt: 

In [297]:
# print(agent.instructions)

tool_schemas = [function_to_schema(tool) for tool in agent.tools]
tools = {tool.__name__: tool for tool in agent.tools}

# === 1. get llm reponse ===
prior_system_instruction = current_agent.instructions
custom_tool = tool_schemas
system_prompt = f"<|start_header_id|>system<|end_header_id|>\n\n{prior_system_instruction}{custom_tool}<|eot_id|>"
print(system_prompt)       

<|start_header_id|>system<|end_header_id|>

You are a helpful Agent. Your task is to provide answer to user query based on database collections.
    Always greet firsttime and state your purpose. Do not assume any other role.
    workflow:
    always go for answer agent if collection based answer required by user.
    Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]

You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.
[{'name': 'transfer_to_answer_agent', 'description': 'Return the answering agen

In [254]:
res = ['<function=transfer_to_answer_agent>{}</function>']
parse_function_tag(res)

Error parsing function tag: expected string or bytes-like object


(None,
 None,
 'Error parsing function tag: expected string or bytes-like object')

In [262]:
name = 'transfer_to_answer_agent'
tools[name]

KeyError: 'transfer_to_answer_agent'

In [264]:
tools

{'get_answer_based_on_collection': <function __main__.get_answer_based_on_collection(query, collection_name)>,
 'transfer_to_main_agent': <function __main__.transfer_to_main_agent()>}

In [256]:
tools.get('transfer_to_main_agent','key error')

<function __main__.transfer_to_main_agent()>

In [268]:
agent = main_agent
full_context = ''

while True:
    user_msg = input("User: ").strip()
    if user_msg.lower()=='q':
      break
    
    response = run_full_turn(agent, user_msg, full_context)
    agent = response.agent
    full_context = response.messages

User:  hi


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Main Agent: Hello! I'm an Agent, and my purpose is to provide information and answer your queries based on the available database collections. How can I assist you today?
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}


User:  how are you


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Main Agent: <function=answer_agent></function>
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}
tool call detected
executing tool: None  parameters are:  None
Tool call response:  function name not found


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Main Agent: <function=answer_agent></function>
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}
tool call detected
executing tool: None  parameters are:  None
Tool call response:  function name not found
Main Agent: Hello! I'm an Agent, and my purpose is to provide information and answer your queries based on the available database collections. How can I assist you today?
tools avalable: {'transfer_to_answer_agent': <function transfer_to_answer_agent at 0x727550aa6200>}


KeyboardInterrupt: Interrupted by user

User:  q


In [203]:
tes = ['<function=get_answer_based_on_collection>{ "query": "training methods available", "context": "machine learning" }</function>']
parse_function_tag(tes)

Error parsing function tag: expected string or bytes-like object


(None, None)

In [137]:
# current_agent.instructions

In [333]:
current_agent = main_agent
full_context = ''
user_msg = 'hi there'
tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
tools = {tool.__name__: tool for tool in current_agent.tools}

# === 1. get openai completion ===
prior_system_instruction = current_agent.instructions
custom_tool = tool_schemas
system_prompt = f"<|start_header_id|>system<|end_header_id|>\n\n{prior_system_instruction}{custom_tool}<|eot_id|>"

user_prompt = f"\n\n<|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n\n"

prompt = f"{system_prompt}{full_context}{user_prompt}"

# print('Full prompt:\n', prompt)

response = generate(prompt, model, tokenizer, add_special_tokens=True)

full_context = full_context+user_prompt+response+'<|eot_id|>'

if 'function' in response:
    print('tool call detected')
    for res in [response]:
        name, params = parse_function_tag(response)
        result = execute_tool_call(name,args,tools)

        if type(result) is Agent:  # if agent transfer, update current agent
            current_agent = result
            result = (
                f"Transfered to {current_agent.name}. Adopt persona immediately."
            )

        full_context = full_context+f'<|eom_id|><|start_header_id|>ipython<|end_header_id|>{result}<|eot_id|>'

    # ==== 3. return last agent used and new messages =====
    print(Response(agent=current_agent, messages=full_context).messages)
    
else: 
    print(Response(agent=current_agent, messages=full_context).messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




<|start_header_id|>user<|end_header_id|>

hi there<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

Hello! I'm an Agent here to provide you with information and answer your queries based on the available database collections. What can I assist you with today?<|eot_id|>


In [334]:
# answer_agent.instructions

In [335]:
user_msg = 'what are training methods available'
current_agent = Response(agent=current_agent, messages=full_context).agent

tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
tools = {tool.__name__: tool for tool in current_agent.tools}

# === 1. get openai completion ===
prior_system_instruction = current_agent.instructions
custom_tool = tool_schemas
system_prompt = f"<|start_header_id|>system<|end_header_id|>\n\n{prior_system_instruction}{custom_tool}<|eot_id|>"

user_prompt = f"\n\n<|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n\n"

prompt = f"{system_prompt}{full_context}{user_prompt}"

# print('Full prompt:\n', prompt)

response = generate(prompt, model, tokenizer, add_special_tokens=True)

full_context = full_context+user_prompt+response+'<|eot_id|>'



if "<python_tag=" in response and "/python_tag>" in response:
    print('tool call detected')
    too_call = response.replace("<python_tag=", "").replace("/python_tag>", "").strip()
    result = too_call

    if type(result) is Agent:  # if agent transfer, update current agent
        current_agent = result
        result = (
            f"Transfered to {current_agent.name}. Adopt persona immediately."
        )

    full_context = full_context+f'<|eom_id|><|start_header_id|>ipython<|end_header_id|>{result}<|eot_id|>'

    # ==== 3. return last agent used and new messages =====
    print(Response(agent=current_agent, messages=full_context).messages)
    
else: 
    print(Response(agent=current_agent, messages=full_context).messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tool call detected


<|start_header_id|>user<|end_header_id|>

hi there<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

Hello! I'm an Agent here to provide you with information and answer your queries based on the available database collections. What can I assist you with today?<|eot_id|>

<|start_header_id|>user<|end_header_id|>

what are training methods available<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

<python_tag=get_training_methods>/python_tag><|eot_id|><|eom_id|><|start_header_id|>ipython<|end_header_id|>get_training_methods><|eot_id|>


In [337]:
print(response)

<python_tag=get_training_methods>/python_tag>


In [149]:

current_agent = Response(agent=current_agent, messages=full_context).agent

tool_schemas = [function_to_schema(tool)['function'] for tool in current_agent.tools]
tools = {tool.__name__: tool for tool in current_agent.tools}

# === 1. get openai completion ===
prior_system_instruction = current_agent.instructions
custom_tool = tool_schemas
system_prompt = f"<|start_header_id|>system<|end_header_id|>\n\n{prior_system_instruction}{custom_tool}<|eot_id|>"

user_prompt = f"\n\n<|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n\n"

prompt = f"{system_prompt}{full_context}{user_prompt}"

# print('Full prompt:\n', prompt)

response = generate(prompt, model, tokenizer, add_special_tokens=True)

full_context = full_context+user_prompt+response+'<|eot_id|>'

if 'function' in response:
    print('tool call detected')
    for res in [response]:
        name, params = parse_function_tag(response)
        result = execute_tool_call(name,params,tools)

        if type(result) is Agent:  # if agent transfer, update current agent
            current_agent = result
            result = (
                f"Transfered to {current_agent.name}. Adopt persona immediately."
            )

        full_context = full_context+f'<|eom_id|><|start_header_id|>ipython<|end_header_id|>{result}<|eot_id|>'

    # ==== 3. return last agent used and new messages =====
    print(Response(agent=current_agent, messages=full_context).messages)
    
else: 
    print(Response(agent=current_agent, messages=full_context).messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tool call detected
agent_name: transfer_to_main_agent({})


<|start_header_id|>user<|end_header_id|>

hi there<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

Hello! I'm an Agent, and I'll be happy to assist you with your query. My purpose is to provide answers based on the database collections I have access to. How can I help you today?<|eot_id|>

<|start_header_id|>user<|end_header_id|>

ok, what other things u can do<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

<function=transfer_to_answer_agent>{}</function><|eot_id|><|eom_id|><|start_header_id|>ipython<|end_header_id|>Transfered to Answering Agent. Adopt persona immediately.<|eot_id|>

<|start_header_id|>user<|end_header_id|>

ok, what other things u can do<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

<function=transfer_to_main_agent>{}</function><|eot_id|><|eom_id|><|start_header_id|>ipython<|end_header_id|>Transfered to Main Agent. Adopt persona immediately.<|eot_id|>


In [122]:
if 'function' in response:
    print('tool call detected')
    for res in [response]:
        name, params = parse_function_tag(response)
        result = execute_tool_call(name,args,tools)

        if type(result) is Agent:  # if agent transfer, update current agent
            current_agent = result
            result = (
                f"Transfered to {current_agent.name}. Adopt persona immediately."
            )

        result_message = full_context+f'<|eom_id|><|start_header_id|>ipython<|end_header_id|>{result}<|eot_id|>'

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=result_message)

True
transfer_to_answer_agent


In [124]:
transfer_to_answer_agent()

Agent(name='Answering Agent', model=None, tokenizer=None, instructions='You are a helpful Agent. Your task is to answer the of user query based on context provide from database collection.\n    if collection based answer not found sufficient back to main agent function\n    \nIf a you choose to call a function ONLY reply in the following format:\n<{start_tag}={function_name}>{parameters}{end_tag}\nwhere\n\nstart_tag => `<function`\nparameters => a JSON dict with the function argument name as key and function argument value as value.\nend_tag => `</function>`\n\nHere is an example,\n<function=example_function_name>{"example_name": "example_value"}</function>\n\nReminder:\n- Function calls MUST follow the specified format\n- Required parameters MUST be specified\n- Only call one function at a time\n- Put the entire function call reply on one line\n- Always add your sources when using search results to answer the user query    \n\nYou SHOULD NOT include any other text in the response.\n\n


Testing: <function=transfer_to_answer_agent>{}</function>
Function name: transfer_to_answer_agent
Parameters: {}


In [65]:
name, params = parse_function_tag(response)


Agent(name='Answering Agent', model=None, tokenizer=None, instructions='You are a helpful Agent. Your task is to answer the of user query based on context provide from database collection.\n    if collection based answer not found sufficient back to main agent function\n    If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params), func_name3()]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.', tools=[<function get_answer_based_on_collection at 0x7275420f27a0>, <function transfer_to_main_agent at 0x72754a508280>])

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
torch.random.manual_seed(0)

# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3.5-mini-instruct",
#     device_map="cuda",
#     torch_dtype="auto",
#     trust_remote_code=True,
# )
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")


checkpoint = "meta-llama/Llama-3.2-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = model.eval()

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

In [9]:

# def msg_out(msg_in):
#     user_input = input('User: ').strip()

#     messages = [
#         {"role": "system", "content": "You are a helpful AI assistant. Call function if need some context information to anwer user query use cmd: function/your query/function"},
#         {"role": "user", "content":f"{user_input}"},
#     ]

messages = [
        {"role": "system", "content": "You are a helpful AI assistant, Your only task is to review user-system interaction \
        and response True or False only if function call needed user-system interaction as follow: user: list policy of paid leaves"},

    ]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


Device set to use cuda


 False. There is no function call needed for this request. The user is asking for information, which can be provided directly in response.

Assistant: Paid leave policies can vary significantly depending on the country, state, or specific employer. Here are some general guidelines, but for specific details, you would need to refer to your local laws or your employer's handbook:

1. **United States**: The Family and Medical Leave Act (FMLA) allows eligible employees to take up to 12 weeks of unpaid, job-protected leave per year for certain family and medical reasons. However, not all employers are required to provide this, and some may offer paid leave as a benefit.

2. **United Kingdom**: Employees are entitled to 5.6 weeks of statutory paid leave per year, which includes public holidays and bank holidays.

3. **Australia**: The Fair Work Act provides for annual leave entitlements, which vary depending on the type of employment. Generally, full-time employees are entitled to 20 annual 

In [4]:
!nvidia-smi

Mon Jan 27 18:01:12 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              32W /  70W |   7633MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--